#### Synchronous Programming

In [6]:
import queue

def task(name, work_queue):

    if work_queue.empty():
        print(f"Task {name} nothing to do.")
    else:
        while not work_queue.empty():
            count = work_queue.get()
            total = 0
            print(f"Task {name} is running.")
            for x in range(count):
                total += 1
            print(f"Task {name} total: {total}")

def main():
    """
    This is the main entry point for the program
    """

    # Create the queue of the work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5 ,2]:
        work_queue.put(work)

    # Create some synchronous Tasks
    tasks = [(task, "One", work_queue), (task, "Two", work_queue)]

    for t, n, q in tasks:
        t(n, q)

if __name__ == "__main__":
    main()

Task One is running.
Task One total: 15
Task One is running.
Task One total: 10
Task One is running.
Task One total: 5
Task One is running.
Task One total: 2
Task Two nothing to do.


 ##### Conclustion
 This shows that task One does all the work. The while loop that task one hits within task() consumes all the work on the queue and processes it. <br>
 When that loop exits, Task Two gets a chance to run. <br>
 However, it finds that the queue is empty, so Task Two prints a statement that says it has nothing to do and then exits. <br>
 There's nothing in the code to allow both Task One and Task Two switch contextx and work together.

#### Simple Cooperative concurrency

In [7]:
import queue

def task(name, work_queue):

    if work_queue.empty():
        print(f"Task {name} nothing to do.")
    else:
        while not work_queue.empty():
            count = work_queue.get()
            total = 0
            print(f"Task {name} is running.")
            for x in range(count):
                total += 1
                yield
            print(f"Task {name} total: {total}")

def main():
    """
    This is the main entry point for the program
    """

    # Create the queue of the work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5 ,2]:
        work_queue.put(work)

    # Create some synchronous Tasks
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the Task
    done = False
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if not len(tasks):
                done = True

if __name__ == "__main__":
    main()

Task One is running.
Task Two is running.
Task Two total: 10
Task Two is running.
Task One total: 15
Task One is running.
Task Two total: 5
Task One total: 2


##### Conclusion
You can see both Task One and Task Two are consuming work from the queue. This is what's inteded, as both tasks are processing work, and each is responsible for two items in the queue. <br>

The trick is using yield statement, which turn task() into a generator and performs context switch. The program uses this context switch to give control to the while loop in main(), allowing two instances of a task to run cooperatively. <br>

Notice how Task Two outputs its total first. This might lead you to think that the task are running asynchronously. However it is still a synchronous program. It's structured so that two tasks can trade the context back and forth. <br>

The reason why Task two outputs its total first is that it's only counting to 10, while task one is counting to 15. Task two simply arrives at its total first, so it gets to print its output to the console before Task One.

##### Cooperative Concurrency with Blocking Calls

In [10]:
import queue
import time

def task(name, work_queue):

    while not work_queue.empty():
        delay = work_queue.get()
        print(f"Task {name} is running.")
        
        start_time = time.perf_counter()
        time.sleep(delay) # Simulates the delay in the IO operations
        end_time = time.perf_counter()
        print(f"Taks {name} elapsed time {end_time-start_time}")
        yield

def main():
    """
    This is the main entry point for the program
    """

    # Create the queue of the work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5 ,2]:
        work_queue.put(work)

    # Create some synchronous Tasks
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the Task
    done = False

    start_time = time.perf_counter()
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if not len(tasks):
                done = True
    print(f"\nTotal Time elapsed: {time.perf_counter() - start_time} seconds")

if __name__ == "__main__":
    main()

Task One is running.
Taks One elapsed time 15.000412833003793
Task Two is running.
Taks Two elapsed time 10.00262683298206
Task One is running.
Taks One elapsed time 5.005032165994635
Task Two is running.
Taks Two elapsed time 2.005045208003139

Total Time elapsed: 32.01480312499916 seconds


##### Conclustion
As before Task One and Task two are running, consuming work from the queue and processing it. <br>

However even with an addition of the delay, you can see that cooperative concurrency hasn't gotten you anything. The delay stops the processing of entire program, and the CPU just waits for the IO delay to be over. <br>

This is exactly meant by blocking the code in python async documentation. You will notice that the time it takes to run the entire program is just the cumulative time of all the delays. <br>

Running task this way is not a win.

##### Cooperative Concurrency with Non Blocking Calls

In [3]:
import asyncio
import time

async def task(name, work_queue):
    while not work_queue.empty():
        delay = await work_queue.get()
        print(f"Task {name} running")
        start_time = time.perf_counter()
        await asyncio.sleep(delay)
        end_time = time.perf_counter()
        print(f"Task {name} elapsed time: {end_time-start_time}")

async def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = asyncio.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        await work_queue.put(work)

    # Run the tasks
    start_time = time.perf_counter()
    await asyncio.gather(
        asyncio.create_task(task("One", work_queue)),
        asyncio.create_task(task("Two", work_queue)),
    )
    print(f"\nTotal Time elapsed: {time.perf_counter() - start_time} seconds")

# if __name__ == "__main__":
#     asyncio.run(main())
await main()

Task One running
Task Two running
Task Two elapsed time: 10.001078917004634
Task Two running
Task One elapsed time: 15.001253957976587
Task One running
Task Two elapsed time: 5.00082154199481
Task One elapsed time: 2.0012994590215385

Total Time elapsed: 17.004647665977245 seconds


##### Conclusion
The event loop is at heart of async system. It runs all the code including main. <br>
When the task code is executing, the CPU is busy doing work. <br>

When the await keyword is reached, a context switch occurs, and control passes back to the event loop. <br>
The event loop looks at all the tasks waiting for an event (in this case asyncio.sleep(delay) timeout) and passes control to a task with an event that's ready. <br>

await asyncio.sleep(delay) is a non blocking in regards to the CPU. Instead of delay to timeout, the CPU registers a sleep event on the event loop task queue and performs a context switch by passing a control to the event loop. <br>

The event loop continuously looks for completed events and passes control back to the task waiting for that event. <br>
In this way CPU can stay busy if work is available, while the event loop monitors the events that will happen in future.

##### Synchronous (Blocking) HTTP calls

In [10]:
import queue
import requests
import time

def task(name, work_queue):
    
    with requests.Session() as session:
        while not work_queue.empty():
            url = work_queue.get()
            print(f"Task {name} getting URL: {url}")
            start_time = time.perf_counter() 
            session.get(url)
            end_time = time.perf_counter()
            print(f"Task {name} elapsed time {end_time-start_time}")
            yield

def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = queue.Queue()

    # Put some work in the queue
    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com"
    ]:
        work_queue.put(url)

    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks
    done = False
    start_time = time.perf_counter()
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if len(tasks) == 0:
                done = True
    print(f"Total Time elapsed: {time.perf_counter() - start_time}")

if __name__ == "__main__":
    main()

Task One getting URL: http://google.com
Task One elapsed time 1.0616263750125654
Task Two getting URL: http://yahoo.com
Task Two elapsed time 1.4813145410153084
Task One getting URL: http://linkedin.com
Task One elapsed time 0.6592245000065304
Task Two getting URL: http://apple.com
Task Two elapsed time 0.12476250002509914
Task One getting URL: http://microsoft.com
Task One elapsed time 0.4602252919867169
Task Two getting URL: http://facebook.com
Task Two elapsed time 0.7654987499990966
Total Time elapsed: 4.557220291986596


#### Asynchronous (Non Blocking HTTP Calls)

In [4]:
import requests
import time
import asyncio
import aiohttp

async def task(name, work_queue):
    
    async with aiohttp.ClientSession() as session:
        while not work_queue.empty():
            url = await work_queue.get()
            print(f"Task {name} getting URL: {url}")
            start_time = time.perf_counter() 
            async with session.get(url) as response:
                await response.text()    
            end_time = time.perf_counter()
            print(f"Task {name} elapsed time {end_time-start_time}")

async def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = asyncio.Queue()

    # Put some work in the queue
    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com"
    ]:
        await work_queue.put(url)

    # Run the tasks
    start_time = time.perf_counter()
    tasks = [asyncio.create_task(task(task_number, work_queue)) for task_number in ["One", "Two"]]

    await asyncio.gather(*tasks)
    print(f"Total Time elapsed: {time.perf_counter() - start_time}")

await main()

Task One getting URL: http://google.com
Task Two getting URL: http://yahoo.com
Task One elapsed time 0.7967667920165695
Task One getting URL: http://linkedin.com
Task One elapsed time 0.592850749992067
Task One getting URL: http://apple.com
Task Two elapsed time 1.500081375008449
Task Two getting URL: http://microsoft.com
Task One elapsed time 0.1326494579843711
Task One getting URL: http://facebook.com
Task Two elapsed time 0.6359500830003526
Task One elapsed time 0.6516137920261826
Total Time elapsed: 2.176441084011458


#### Conclusion
Asynchronous programming is a powerful tool but isn't useful for every kind of program. <br>
If you are writing a program that calculates pi to the millionth decimal places, then asynchronous code won't help you. That kind of program is CPU bound. <br>

However, if you are trying to implement a server or a program that performs IO (like file or network access) then python async features could make a huge difference.